gitHUB Actions
==============



In [1]:
# Start clean
! rm -fr s23-06682-L08

# Lecture setup

1. Make sure you are logged into GitHUB.
2. Go to https://github.com/Steel-Bank-codespaces/s23-06682-L08.

Look for the fork button.

then click it. This will "fork" the repo, i.e. make a copy of it in your GitHUB account.

Then, come back here and open a terminal. Navigate to the lecture directory and run

    ! git clone https://github.com/<github-id>/s23-06682-L08.git

In [3]:
! git clone https://github.com/jkitchin/s23-06682-L08.git

Cloning into 's23-06682-L08'...
Username for 'https://github.com': ^C


    
to make your own copy locally we can work on.

# Brief exploration of the repo 



In [4]:
! tree s23-06682-L08

s23-06682-L08  [error opening dir]

0 directories, 0 files


In [5]:
%%bash
cd s23-06682-L08
pytest

bash: line 1: cd: s23-06682-L08: No such file or directory


============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.3.2, pluggy-1.0.0
rootdir: /home/jovyan/work/08-github-actions
plugins: anyio-3.6.1, nbmake-1.4.1
collected 0 items

============================ no tests ran in 0.02s =============================


CalledProcessError: Command 'b'cd s23-06682-L08\npytest\n'' returned non-zero exit status 5.

In [ ]:
%%bash
cd s23-06682-L08/pkg
pip install .

In [ ]:
%run s23-06682-L08/example.ipynb

Looks good for now. 



# Making GitHUB do the work instead of your computer

We have learned about pre-commit hooks, where you leverage git to run tests, do formatting, check styles, etc. These are ideal for your side of the work, that is, we check before submitting to GitHUB.

There is also a way to do checks /after/ you submit to GitHUB. This is a specific feature of GitHUB, but you may find similar features in other services like bitbucket or gitlab.

A workflow is a setup for an environment to run some commands in, and a set of commands to be run. Before we get into the details of those two things, we will talk about YAML (https://yaml.org/), which is recursively defined as "YAML ain't markup language". It is a data serialization language often used for configurations. It is specifically a data language, and is not meant to be used to write documents (in contrast to Markdown, for example).

You can learn YAML here https://learnxinyminutes.com/docs/yaml/. We will learn by example today.

Yaml is comprised of key:value pairs (like a dictionary) and lists.  These can be opened/closed by changing indentation levels.

To get a sense for how to read YAML, we will use pyyaml to read some.



In [ ]:
import yaml


yaml.load('name: my action', Loader=yaml.SafeLoader)



A two line string loads like this:



In [ ]:
yaml.load('''
name: grading
runs-on: ubuntu-latest
''', Loader=yaml.SafeLoader)



In [ ]:
yaml.load('''
steps:
  - uses: actions/checkout@v3
    with:
      fetch-depth: 0
  - uses: education/autograding@v1
    name: "Grading and Feedback"
''', Loader=yaml.SafeLoader)



These files usually get put into a file with an extension of .yml or .yaml. We have to create YAML files to tell GitHUB what to do for actions and workflows.




# GitHUB Actions (https://docs.github.com/en/actions)

To make a new workflow, you create a .yml or .yaml file in ~.github/workflows~.

GitHUB actions are sophisticated, and cover a lot of ground:
https://docs.github.com/en/actions/using-workflows/workflow-syntax-for-github-actions#about-yaml-syntax-for-workflows.

There are a few things every workflow must have in it. One is a name, and one is what triggers it to run. Here we specify those things. Put this in a file called `.github/workflows/my-workflow.yaml`. This will give your action a name, and it will get run when ever anything is pushed.



In [ ]:
%%bash
mkdir -p s23-06682-L08/.github/workflows



In [ ]:
%%writefile s23-06682-L08/.github/workflows/my-workflow.yaml
name: First job
on: push

jobs:
  my_first_job:
    runs-on: ubuntu-latest
    steps:
      - run: |
          pwd 
          ls -alF



Go ahead and commit this new file. Before you push it, remember to go to https://github.com/settings/tokens to setup a personal access token. Make sure to select the repo and workflow items this time.

Also, go ahead and run this command in the repo in the terminal before you push:

    git config credential.helper store
    
then, go ahead and push it back to your forked version. 

```{note}
you may see something like

jupyter-jkitchin@andrew.cm-11dd7@jupyterhub-dev:~/src/lectures/08-github-actions/s23-06682-L08$ git push
remote: Invalid username or password.
fatal: Authentication failed for 'https://github.com/jkitchin/s23-06682-L08.git/'

because there may be a saved value from last class. Try it again and see if it prompts you for a username, and then paste your new token in.
```

Assuming that worked, now we can go check out what happened. Navigate back to your repo. You can find it like this:



In [ ]:
%%bash
cd s23-06682-L08
git remote -v



There are a bunch of new things. You can see where the repo was forked from, and that it is one commit ahead of the repo it was forked from.



Next, navigate to the Actions tab. Let's see what happened. 

There isn't much in the actions output. In fact, there is nothing at all! When the virtual machine that runs this spins up, it is bare and has almost nothing in it. It does not even have your repo in it.

Let's add a second job and learn a new trick about using "actions". Here we checkout a copy of our repo, then we update apt so we can install tree. What is with the "sudo"? You need /root/ or /superuser/ privileges to install packages, and we get that with `sudo`. It is short for "superuser do".



In [ ]:
%%writefile s23-06682-L08/.github/workflows/my-workflow.yaml
name: First job
on: push

jobs:
  my_first_job:
    runs-on: ubuntu-latest
    steps:
      - run: |
          pwd 
          ls -alF

  my_second_job:
    runs-on: ubuntu-latest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          sudo apt-get update 
          sudo apt-get install tree 
          tree -F .            



Now we need to commit and push this.



In [ ]:
%%bash
cd s23-06682-L08
git commit .github/workflows/my-workflow.yaml -m "add second job"
git push



It may take a while, but soon a new Action task should launch.

Finally, you see we have a copy of our current repository. This still isn't that exciting, after all, we already knew we had those! We are leading up to setting up some tests. 

It is a little challenging to know what all is installed remotely. Let's add a third job, this time we install `black` and run it on our repository. We only run in check mode, so if this fails, then this job will fail.

We don't need the first or second job any more, so let's delete that.



In [ ]:
%%writefile s23-06682-L08/.github/workflows/my-workflow.yaml
name: First job
on: push

jobs:

  my_third_job:
    runs-on: ubuntu-latest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          pip install black
          black --check .



In [ ]:
%%bash
cd s23-06682-L08
git commit .github/workflows/my-workflow.yaml -m "add third job"
git push



Now go back to your Actions tab. You will see that the third job should have failed. We can fix it by running black locally. Check your email, you should have gotten a notification that a job failed!



In [ ]:
%%bash
cd s23-06682-L08
black .
git commit -am "ran black"
git push



## Create a status badge

In the actions view there is a place to create a status badge. This is some Markdown that GitHUB can render. Copy this to your Readme.md file, then commit and push it.



In [ ]:
%%bash
cd s23-06682-L08
git commit README.md -m "add badge"
git push



Now on gitHUB, you should be able to see the badge.



Let's add one new task to run our tests. This one requires us to install all the libraries needed for our test to work. Note that with the checkout action we start in our repo.



In [ ]:
%%writefile s23-06682-L08/.github/workflows/my-workflow.yaml
name: First job
on: push

jobs:

  black:
    runs-on: ubuntu-latest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          pip install black
          black --check .
        
  pytest:
    runs-on: ubuntu-latest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          pip install matplotlib requests IPython pytest
          pytest .



In [ ]:
%%bash
cd s23-06682-L08
git commit -am "add black and pytest with better names"
git push



# Cross platform testing

These tests are more expensive than the Linux tests, so do not use them alot if you don't need them.



## Testing on Windows.



In [ ]:
%%writefile s23-06682-L08/.github/workflows/windows-workflow.yaml
name: Windows testing
on: push

jobs:
  windows:
    runs-on: windows-latest
    steps:
      - run: |
          systeminfo
        
  pytest:
    runs-on: windows-lattest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          pip install matplotlib requests IPython pytest
          pytest .        



In [ ]:
%%bash
cd s23-06682-L08
git add .github/workflows/windows-workflow.yaml
git commit -m "add Windows test"
git push



## Testing for Macs



In [ ]:
%%writefile s23-06682-L08/.github/workflows/mac-workflow.yaml
name: MacOS testing
on: push

jobs:
  mac:
    runs-on: macos-latest
    steps:
      - run: |
          pwd 
          ls -alF
          sw_vers
        
  pytest:
    runs-on: macos-latest
    steps:      
      - uses: actions/checkout@v3
      - run: |
          pip install matplotlib requests IPython pytest
          python -m pytest       



In [ ]:
%%bash
cd s23-06682-L08
git add .github/workflows/mac-workflow.yaml
git commit -m "add Mac test"
git push



# Custom environments with Docker

It can be tedious, and time-consuming to have to install everything in your GitHUB actions environment. The tedious part is not too avoidable, that work has to be done somewhere, at least once. The time-consuming part can be problematic though; 1) you have to wait for it each time, 2) for open-source projects the time is "free" but may be capped at 2000 minutes a month (or whatever GitHUB decides it wants). Furthermore, a single workflow can only run up to 58 minutes. There are three strategies to mitigate these: 1) Pay for more time, 2) setup an external server to run your actions, and 3) streamline the actions by using Docker images that are pre-built. We only consider the last of these today, and we only partially consider how to use Docker.

This example is adapted from https://docs.github.com/es/actions/creating-actions/creating-a-docker-container-action.

This is an advanced example. It may be the longest version of "Hello World" I have seen yet. There is quite a bit of setup here. The idea is we run a virtual machine inside the virtual machine. Here we use a Dockerfile inside our repo, but you can also use images from Dockerhub, for example. 

Our goal is to run a simple shell script inside a Docker image that is inside GitHUB actions. This allows us to customize the environment, e.g. we could install python, or LaTeX, or any other things we want. We don't do those things here, and instead focus on the smallest working example we can get get.

First, we define a shell script called entrypoint.sh. It will take one argument, and echo "Hello argument". We will also run our tests in this script. Make this script executable. 



In [ ]:
%%writefile s23-06682-L08/entrypoint.sh
#!/bin/sh -l

echo "Hello $1"
pwd
ls
pytest



In [ ]:
! chmod +x s23-06682-L08/entrypoint.sh



Next we make the Dockerfile. We have to provide directions in this file to setup our environment. Docker is complicated, and we won't get all the way into it. The gist here is we start with a base image `python3-slim`, and then install software in it, and copy our script into it. GitHUB will build the image, and use it for your actions. The entrypoint.sh script is run when the image starts up.



In [ ]:
%%writefile s23-06682-L08/Dockerfile
# Container image that runs your code
FROM python:3-slim 

RUN pip3 install --upgrade pip setuptools wheel matplotlib requests IPython pytest

# Copies your code file from your action repository to the filesystem path `/` of the container
COPY entrypoint.sh /entrypoint.sh
    
# Code file to execute when the docker container starts up (`entrypoint.sh`)
ENTRYPOINT ["sh", "/entrypoint.sh"]



Next, we set up a GitHUB *action*. This is a yaml file saved as `action.yml` in your repo root. This is a little different than the workflows above. This defines some inputs that will be used and passed to the Docker image.

The syntax ~${{ inputs.who-to-greet }}~ is /like/ a python f-string, but in yaml instead. The value of "who-to-greet" will be used in the template string.



In [ ]:
%%writefile  s23-06682-L08/action.yml
name: 'Hello World'
description: 'Greet someone and record the time'
inputs:
  who-to-greet:  # id of input
    description: 'Who to greet'
    required: true
    default: 'World'

runs:
  using: 'docker'
  image: 'Dockerfile'
  args:
    - ${{ inputs.who-to-greet }}



Finally, we define a workflow to use our action. We specify a "path" to our action, which is of the form `<user>/<repo>@<version>`. For version we use the main branch, which should always use the latest version. Next we define the input we want using `with`. The name of this variable matches the name used in the inputs of the action. Put this into `.github/workflows/try-docker.yaml`.



In [ ]:
%%writefile s23-06682-L08/.github/workflows/try-docker.yaml
name: Try Docker
on: push

jobs:
  run-my-docker:
    runs-on: ubuntu-latest
    steps:      
      - uses: actions/checkout@v3
      - uses: jkitchin/s23-06682-L08@main
        with:
          who-to-greet: John




Finally, push these out to GitHUB.



In [ ]:
%%bash
cd s23-06682-L08
git add entrypoint.sh Dockerfile action.yml .github/workflows/try-docker.yaml
git commit -m "added docker"
git push



When you push this, you will see in the actions output, "Hello John", and some other output for the tests.

This may seem like quite some overkill to print "Hello John". The real value of this comes when you start using files in the repo as input to do more sophisticated analysis.

Finally, this Docker image is built at run time. This is not the fastest way to do this. A faster way is to register the images on DockerHUB, and then you just use the image directly.

There is *so much* flexibility in Docker images. Their main limitation is they are Linux based.



# Cleaning up

You can delete your GitHUB repo now if you want. Go to the settings tab (something like https://github.com/jkitchin/s23-06682-L08/settings). Scroll to the bottom, and click `Delete this repository`. Don't worry, you can re-fork it.

